# Sampling in DGL

As the graph scales up to billions of nodes or edges, training on the full graph would no longer be efficient or even feasible.

Mini-batch training allows us to control the computation and memory usage within some budget. The training loss for each iteration is

$$\frac{1}{\vert \tilde{\mathcal{V}}_\mathcal{L} \vert} \sum_{v \in \tilde{\mathcal{V}}_\mathcal{L}} f(y_v, z_v^{(L)})$$

where $\tilde{\mathcal{V}}_\mathcal{L}$ is a subset sampled from the total labeled nodes $\mathcal{V}_\mathcal{L}$ uniformly at random.

Stemming from the labeled nodes $\tilde{\mathcal{V}}_\mathcal{L}$ in a mini-batch and tracing back to the input forms a computational dependency graph (a directed acyclic graph or DAG in short), which captures the computation flow of $Z^{(L)}$.

In the example below, a mini-batch to compute the hidden features of node D in layer 2 requires hidden features of A, B, E, G in layer 1, which in turn requires hidden features of C, D, F in layer 0.

![alt text](https://s3.us-east-2.amazonaws.com/dgl.ai/tutorial/sampling/NodeFlow.png)

For that purpose, we define ``NodeFlow`` to represent this computation flow.

``NodeFlow`` is a type of layered graph, where nodes are organized in $L + 1$ sequential *layers*, and edges only exist between adjacent layers, forming *blocks*. We construct ``NodeFlow`` backwards, starting from the last layer with all the nodes whose hidden features are requested. The set of nodes the next layer depends on forms the previous layer. An edge connects a node in the previous layer to another in the next layer iff the latter depends on the former. We repeat such process until all $L + 1$ layers are constructed. The feature of nodes in each layer, and that of edges in each block, are stored as separate tensors.

``NodeFlow`` provides ``block_compute`` for per-block computation, which triggers computation and data propogation from the lower layer to the next upper layer.

## Neighbor sampling

Instead of using all the $L$-hop neighbors of a node $v$, neighbor sampling randomly samples a few neighbors $\hat{\mathcal{N}}^{(l)}(v)$ to estimate the aggregation $z_v^{(l)}$ of its total neighbors $\mathcal{N}(v)$ in $l$-th GCN layer by an unbiased estimator $\hat{z}_v^{(l)}$

$$
\hat{z}_v^{(l)} = \frac{\vert \mathcal{N}(v) \vert }{\vert \hat{\mathcal{N}}^{(l)}(v) \vert} \sum_{u \in \hat{\mathcal{N}}^{(l)}(v)} \tilde{A}_{uv} \hat{h}_u^{(l)} \\
\hat{h}_v^{(l+1)} = \sigma ( \hat{z}_v^{(l)} W^{(l)} )
$$

Let $D^{(l)}$ be the number of neighbors to be sampled for each node at the $l$-th layer, then the receptive field size of each node can be controlled under $\prod_{i=0}^{L-1} D^{(l)}$ by neighbor sampling.

## Random-walk sampling

Given a source node $s$ and a decay factor $\alpha$, a random walk from $s$ is a trace beginning from $s$, at each step either proceeds to a neighbor uniformly at random, or stop with probability $\alpha$. The personalized PageRank (PPR) score $\pi(s, t)$ is the probability that a random walk from $s$ terminates at $t$.

In PinSage, each node $v$ selects top-$k$ important nodes with highest PPR scores with respective to $v$ as its neighbors $\hat{\mathcal{N}}(v)$, and the hidden feature of each neighbor $u \in \hat{\mathcal{N}}(v)$ is weighted by $\pi(v, u)$,

$$
\hat{z}^{(l)}_v = \frac{\sum_{u \in \hat{\mathcal{N}}(v)} \pi(v, u) \tilde{A}_{uv} \hat{h}_u^{(l)}}{\sum_{u \in \hat{\mathcal{N}}(v)} \pi(v, u)}
$$

Compared to GCNs which uses all the $L$-hop neighbors, PinSage selects $k$ topology-based important neighbors which have the largest influence.
